### Experiments on the Library of Triangulations

https://www3.math.tu-berlin.de/IfM/Nachrufe/Frank_Lutz/stellar/library_of_triangulations/

In [7]:
!sage --preparse Flow_Category/Flow2.sage
!mv Flow_Category/Flow2.sage.py Flow_Category/Flow2.py
!sage --preparse Flow_Category/Localization.sage
!mv Flow_Category/Localization.sage.py Flow_Category/Localization.py
!sage --preparse Flow_Category/Matchings.sage
!mv Flow_Category/Matchings.sage.py Flow_Category/Matchings.py

from Flow_Category.Flow2 import *
from Flow_Category.Localization import *
from Flow_Category.Matchings import *

In [8]:
import numpy as np
from tqdm import tqdm
import os
import time

We analize minimal regular CW-decompositions of 2-dimensional complexes in the library

In [9]:
triangulations = [
    "Abalone",
    "BH",
    "BH_3",
    "BH_4",
    "BH_5",
    "d2_n8_3torsion",
    "d2_n8_4torsion",
    "d2_n9_5torsion",
    "dunce_hat",
    "d2n12g6",
    "regular_2_21_23_1",
    "rand2_n25_p0.328",
    "dunce_hat_in_3_ball",
    "Barnette_sphere",
    "B_3_9_18",
    "trefoil_arc",
    "trefoil",
    "rudin",
    "poincare",
    "double_trefoil",
    "triple_trefoil_arc",
    "triple_trefoil",
    "hyperbolic_dodecahedral_space",
    "S_3_50_1033",
    "600_cell",
    "CP2",
    "RP4",
    "K3_16",
    "K3_17",
    "RP4_K3_17",
    "RP4_11S2xS2",
    "SU2_SO3",
    "non_PL",
    "bing", "non_4_2_colorable", "Hom_C5_K4", "trefoil_bsd", "knot"
]

In [14]:
two_dim = []
base_directory = "Library_Triangulations"
os.makedirs(base_directory, exist_ok=True)

for s_complex in triangulations:
        print(f"Analysing {s_complex}...")  # Display the current complex name
        
        # Construct file path
        file_path = os.path.join(base_directory, f"{s_complex}.txt")
        
        # Load simplices
        simplices = np.loadtxt(file_path, dtype=int)
        
        # Initialize results

        K = SimplicialComplex(simplices)
        X = K.face_poset()
        if X.height()==3:
            two_dim.append(s_complex)

Analysing Abalone...
Analysing BH...
Analysing BH_3...
Analysing BH_4...
Analysing BH_5...
Analysing d2_n8_3torsion...
Analysing d2_n8_4torsion...
Analysing d2_n9_5torsion...
Analysing dunce_hat...
Analysing d2n12g6...
Analysing regular_2_21_23_1...
Analysing rand2_n25_p0.328...
Analysing dunce_hat_in_3_ball...
Analysing Barnette_sphere...
Analysing B_3_9_18...
Analysing trefoil_arc...
Analysing trefoil...
Analysing rudin...
Analysing poincare...
Analysing double_trefoil...
Analysing triple_trefoil_arc...
Analysing triple_trefoil...
Analysing hyperbolic_dodecahedral_space...
Analysing S_3_50_1033...
Analysing 600_cell...
Analysing CP2...
Analysing RP4...
Analysing K3_16...
Analysing K3_17...
Analysing RP4_K3_17...
Analysing RP4_11S2xS2...
Analysing SU2_SO3...
Analysing non_PL...
Analysing bing...
Analysing non_4_2_colorable...
Analysing Hom_C5_K4...
Analysing trefoil_bsd...
Analysing knot...


In [12]:
def reduction(triangulations, n_sim):
    
    results = []
    procedure_times = []
    
    base_directory = "Library_Triangulations"
    os.makedirs(base_directory, exist_ok=True)
    
    for s_complex in triangulations:
        print(f"Processing {s_complex}...")  # Display the current complex name
        
        # Start timing
        start_time = time.time()
        
        # Construct file path
        file_path = os.path.join(base_directory, f"{s_complex}.txt")
        
        # Load simplices
        simplices = np.loadtxt(file_path, dtype=int)
        
        # Initialize results
        random_reduction = [s_complex]
        reduction_times = [s_complex]
        K = SimplicialComplex(simplices)
        X = K.face_poset()
        random_reduction.append(len(X))
        
        # Containers for statistics and timing
        N_flow_Morse = []

        
        time_flow = []
        
        # Progress bar for simulations
        for i in tqdm(range(1, n_sim + 1), desc=f"Simulations for {s_complex}"):
        
            # Perform Flow Morse reduction
            step_start = time.time()
            M = regular_acyclic_matching(X)
            C = critical_poset(X,M)
            N_flow_Morse.append(len(C))
            step_end = time.time()
            time_flow.append(step_end - step_start)
        
        
        # Append statistics to random_reduction
        random_reduction.append(np.mean(N_flow_Morse))
        reduction_times.append(np.mean(time_flow))     
        
        # End timing
        end_time = time.time()
        
        # Calculate elapsed time
        elapsed_time = end_time - start_time
        
        # Display results in a table-like format
        print(f"\nResults for {s_complex}:\n")
        
        # Fixed width for columns
        label_width = 30  # Width for the label column
        value_width = 12  # Width for the value column
        
        # Header row
        print(f"{'Reduction':<{label_width}} {'Simplices':>{value_width}} {'Time (seconds)':>{value_width}}")
        
        # Separator
        print(f"{'-' * label_width} {'-' * value_width} {'-' * value_width}")
        
        # Table rows for the different procedures
        print(f"{'Original simplicial complex:':<{label_width}} {len(X):>{value_width}} {'-':>{value_width}}")
        print(f"{'Flow Morse core:':<{label_width}} {np.mean(N_flow_Morse):>{value_width}.2f} {np.mean(time_flow):>{value_width}.4f}")

        print(f"Total time taken: {elapsed_time:.2f} seconds\n")
        
        results.append(random_reduction)
        procedure_times.append(reduction_times)
    return results, procedure_times

We perform 10 random simulations of _regular_ acyclic matchings on the face poset of each complex, satysfying that the resulting flow posets are contractible. We then compute the average number of cells in the corresponding critical poset. In all cases, we observe a significant reduction in the number of cells compared to the original triangulation.

In [13]:
n_sim = 10
results, procedure_times = reduction(two_dim, n_sim)

Processing Abalone...


Simulations for Abalone: 100%|██████████| 10/10 [00:17<00:00,  1.72s/it]



Results for Abalone:

Reduction                         Simplices Time (seconds)
------------------------------ ------------ ------------
Original simplicial complex:            101            -
Flow Morse core:                      33.20       1.7187
Total time taken: 17.27 seconds

Processing BH...


Simulations for BH: 100%|██████████| 10/10 [00:27<00:00,  2.74s/it]



Results for BH:

Reduction                         Simplices Time (seconds)
------------------------------ ------------ ------------
Original simplicial complex:            131            -
Flow Morse core:                      40.20       2.7407
Total time taken: 27.42 seconds

Processing BH_3...


...


Results for d2n12g6:

Reduction                         Simplices Time (seconds)
------------------------------ ------------ ------------
Original simplicial complex:            122            -
Flow Morse core:                      47.40       2.1627
Total time taken: 21.64 seconds

Processing regular_2_21_23_1...


Simulations for regular_2_21_23_1: 100%|██████████| 10/10 [01:48<00:00, 10.85s/it]



Results for regular_2_21_23_1:

Reduction                         Simplices Time (seconds)
------------------------------ ------------ ------------
Original simplicial complex:            266            -
Flow Morse core:                      98.20      10.8536
Total time taken: 108.55 seconds

Processing rand2_n25_p0.328...


Simulations for rand2_n25_p0.328: 100%|██████████| 10/10 [54:43<00:00, 328.33s/it]


Results for rand2_n25_p0.328:

Reduction                         Simplices Time (seconds)
------------------------------ ------------ ------------
Original simplicial complex:           1076            -
Flow Morse core:                     736.20     328.3283
Total time taken: 3283.34 seconds

